In [1]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
)

In [2]:
dataset = load_dataset("squad_v2")

README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [6]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# 3. Preprocess the data
max_length = 512  # Maximum length of input sequences
doc_stride = 128  # Helps in splitting long documents

In [8]:
def preprocess_data(examples):
    # Tokenize the questions and context
    encoding = tokenizer(
        examples['question'],
        examples['context'],
        truncation=True,
        padding="max_length",
        max_length=max_length,
        stride=doc_stride,
        return_tensors="pt"
    )

    # Find the start and end positions of the answer in the context
    start_positions = []
    end_positions = []

    for context, answer in zip(examples['context'], examples['answers']):
        if len(answer['text']) > 0:
            # Get the text of the first answer (assuming one answer per question)
            answer_text = answer['text'][0]
            start_position = context.find(answer_text)
            
            # Ensure the answer exists in the context
            if start_position != -1:
                end_position = start_position + len(answer_text) - 1
            else:
                start_position = 0
                end_position = 0
        else:
            # No valid answer
            start_position = 0
            end_position = 0

        start_positions.append(start_position)
        end_positions.append(end_position)

    # Add start and end positions to the encoding
    encoding['start_positions'] = start_positions
    encoding['end_positions'] = end_positions

    return encoding

In [9]:
# Preprocess both train and validation datasets
train_dataset = dataset["train"].map(preprocess_data, batched=True)
val_dataset = dataset["validation"].map(preprocess_data, batched=True)

# Remove unnecessary columns after processing
train_dataset = train_dataset.remove_columns(["question", "context"])
val_dataset = val_dataset.remove_columns(["question", "context"])

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [10]:
training_args = TrainingArguments(
    output_dir="./distilbert-qa",        # Directory to store the model
    evaluation_strategy="epoch",         # Evaluate at the end of each epoch
    learning_rate=2e-5,                  # Learning rate for fine-tuning
    per_device_train_batch_size=8,       # Adjust to fit your GPU memory
    per_device_eval_batch_size=8,        # Same as above for evaluation
    num_train_epochs=3,                  # Number of training epochs
    save_strategy="epoch",               # Save model at the end of each epoch
    save_total_limit=2,                  # Limit the number of saved models
    fp16=True,                           # Enable mixed precision for faster training
    logging_dir="./logs",                # Directory for logging
    logging_steps=100,                   # Log every 100 steps
    report_to="none",                   # Avoid using default WandB or TensorBoard
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
# 5. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

/tmp/ipykernel_23/2415645228.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,3.670500,2.979045
2,3.410200,2.904812
3,3.207600,2.943974


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=24435, training_loss=3.5678972463215546, metrics={'train_runtime': 11510.4471, 'train_samples_per_second': 33.965, 'train_steps_per_second': 2.123, 'total_flos': 5.107974402921062e+16, 'train_loss': 3.5678972463215546, 'epoch': 3.0})

In [13]:
# Evaluate the model on the validation dataset
eval_results = trainer.evaluate()

print("Evaluation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value}")

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Evaluation Results:
eval_loss: 2.943974018096924
eval_runtime: 105.378
eval_samples_per_second: 112.671
eval_steps_per_second: 7.051
epoch: 3.0


In [16]:
inputs = tokenizer(
    test_data["question"],
    test_data["context"],
    truncation=True,
    padding="max_length",
    max_length=max_length,
    return_tensors="pt"
).to("cuda")  # Send the data to GPU if available

In [17]:
# Get the model's predictions
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    outputs = model(**inputs)

# Extract the start and end logits
start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Get the most probable start and end positions
start_index = torch.argmax(start_logits, dim=1).item()
end_index = torch.argmax(end_logits, dim=1).item()

In [19]:
model.save_pretrained("./fine_tuned_distilbert")
tokenizer.save_pretrained("./fine_tuned_distilbert")

('./fine_tuned_distilbert/tokenizer_config.json',
 './fine_tuned_distilbert/special_tokens_map.json',
 './fine_tuned_distilbert/vocab.txt',
 './fine_tuned_distilbert/added_tokens.json',
 './fine_tuned_distilbert/tokenizer.json')

In [20]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

model_path = "./fine_tuned_distilbert"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path).to("cuda")

In [ ]:
def answer_question(question, context):
    # Tokenize the inputs
    inputs = tokenizer(
        question,
        context,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    ).to("cuda")  # Send to GPU if available

    # Get model predictions
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract start and end logits
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Find the most probable start and end positions of the answer
    start_index = torch.argmax(start_logits, dim=1).item()
    end_index = torch.argmax(end_logits, dim=1).item()

    # Handle cases where the model predicts no answer
    if start_index >= end_index or start_index < 0 or end_index < 0:
        return "No answer found."

    # Decode the answer from the token indices
    tokens = inputs["input_ids"][0]
    answer_tokens = tokens[start_index : end_index + 1]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    return answer

# Example usage
question = "When did Beyonce start becoming popular?"
context = 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy"'

answer = answer_question(question, context)
print(f"Question: {question}")
print(f"Answer: {answer}")
